In [1]:
import pandas as pd
import sqlite3 

# 1. Load Dataset

In [2]:
df = pd.read_excel('../dataset/flowers.xlsx')

# 2. Create Sqllite Database

## 2.1 Create Tables

In [3]:
import sqlite3

database = 'flowers.db'

create_table_customer_query = """ 
CREATE TABLE Customer (
    CustomerId TEXT PRIMARY KEY,
    CustomerName TEXT NOT NULL,
    Email TEXT NOT NULL,
    Password TEXT NOT NULL
)
"""

create_table_product_query = """ 
CREATE TABLE Product (
    ProductId TEXT PRIMARY KEY,
    ProductName TEXT NOT NULL,
    Price INTEGER NOT NULL
)
"""

create_table_cartlist_query = """ 
CREATE TABLE Cartlist (
    CustomerId TEXT NOT NULL,
    ProductId TEXT NOT NULL,
    Quantity INTEGER NOT NULL,
    TotalPrice FLOAT NOT NULL,
    FOREIGN KEY (CustomerId) REFERENCES Customer (CustomerId),
    FOREIGN KEY (ProductId) REFERENCES Product (ProductId),
    PRIMARY KEY (CustomerId, ProductId)
);
"""

create_table_query_list = [create_table_customer_query, create_table_product_query, create_table_cartlist_query]

for query in create_table_query_list:
    try:
        with sqlite3.connect(database) as conn:
            cursor = conn.cursor()
            cursor.execute(query)   
            conn.commit()

    except sqlite3.OperationalError as e:
        print(e)

## 2.2 Insert Dummy Data

### 2.2.1 Insert Customer Data

In [4]:
insert_customer_query = """
INSERT INTO Customer (CustomerId, CustomerName, Email, Password)
VALUES (?, ?, ?, ?)
"""
customer_data = [
    ("C001", "Bale", "bale@example.com", "password"),
    ("C002", "Alice", "alice@example.com", "password"),
    ("C003", "Bob", "bob@example.com", "password")
]

with sqlite3.connect(database) as conn:
    cursor = conn.cursor()
    cursor.executemany(insert_customer_query, customer_data)
    conn.commit()
    print("Customer data inserted successfully")


Customer data inserted successfully


### 2.2.2 Insert Product Data

In [5]:
product_data = []
for index, row in df.iterrows():
    product_id = row['Product_Id']
    product_name = row['Product_Name']
    price = row['Price_RM']
    product_data.append((product_id, product_name, price))


In [6]:
insert_product_query = """
INSERT INTO Product (ProductId, ProductName, Price)
VALUES (?, ?, ?)
"""

with sqlite3.connect(database) as conn:
    cursor = conn.cursor()
    cursor.executemany(insert_product_query, product_data)
    conn.commit()
    print("Product data inserted successfully")


Product data inserted successfully


### 2.2.3 Insert CartList Data

In [10]:
insert_cart_query = """
INSERT INTO CartList (CustomerId, ProductId, Quantity, TotalPrice)
VALUES (?, ?, ?, ?)
"""

cart_data = [
    ("C001", "P001", 1, 420),
    ("C002", "P002", 1, 95),
    ("C003", "P003", 1, 280),
    ("C001", "P004", 2, 600),
    ("C002", "P005", 2, 800),
    ("C003", "P006", 2, 520),
]

with sqlite3.connect(database) as conn:
    cursor = conn.cursor()
    cursor.executemany(insert_cart_query, cart_data)
    conn.commit()
    print("Cart data inserted successfully")


Cart data inserted successfully


# Final: Query with the database

In [2]:
import sqlite3
database = 'flowers.db'

conn = sqlite3.connect(database)
cur = conn.cursor()
cur.execute("SELECT * FROM Cartlist WHERE CustomerId == 'C001' ")
rows = cur.fetchall()
conn.close()
for row in rows:
   print(row)

('C001', 'P001', 3, 1260.0)
('C001', 'P003', 2, 560.0)
('C001', 'P004', 2, 600.0)
('C001', 'P009', 4, 400.0)
('C001', 'P015', 3, 600.0)
('C001', 'P021', 6, 540.0)
('C001', 'P023', 10, 5000.0)


In [11]:
def add_item_to_cart_by_name(customer_id, product_name, quantity):
    try:
        with sqlite3.connect('flowers.db') as conn:
            cursor = conn.cursor()
            
            # Get product price and ID
            cursor.execute("SELECT ProductId, Price FROM Product WHERE ProductName = ?", (product_name,))
            product = cursor.fetchone()
            
            if product is None:
                raise ValueError("Product not found")
            
            product_id, price = product
            total_price = price * quantity
            
            # Add or update item in cart
            query = """
            INSERT INTO Cartlist (CustomerId, ProductId, Quantity, TotalPrice)
            VALUES (
                ?,
                ?,
                ?,
                ?
            )
            ON CONFLICT(CustomerId, ProductId)
            DO UPDATE SET
                Quantity = Quantity + excluded.Quantity,
                TotalPrice = (Quantity + excluded.Quantity) * (SELECT Price FROM Product WHERE ProductId = excluded.ProductId);
            """
            cursor.execute(query, (customer_id, product_id, quantity, total_price))
            conn.commit()

            return {
                "process_message":"Item added to cart successfully.",
                "completed":True
            }
        
    except sqlite3.Error as e:
        print(f"Error: {e}")
        return {
                "process_message":"Fail Add to Cart",
                "completed":False
            }
    except ValueError as ve:
        print(ve)
        return {
                "process_message":"Item added to cart successfully.",
                "completed":False
            }



In [13]:
add_item_to_cart_by_name('C001', 'Elsa Rose Bouquet', 2)


{'process_message': 'Item added to cart successfully.', 'completed': True}